In [13]:
# Data Preprocessing UnemploymentbyRegion
import pandas as pd
import numpy as np


In [49]:
# EDA on quarterly unemployment from 2004_22
df = pd.read_csv('unemployment_quartely_2004-22_clean.csv')

In [52]:
df.head(100)

,Territory_ID,Territory,Territory_acr,Gender_ID,Gender,ages from,ages to,Year,Quarter,Unemployment rate
0,1,Italy,IT,1,males,15,24,2004.0,Total,20.664027
1,1,Italy,IT,1,males,15,24,2004.0,2.0,21.089639
2,1,Italy,IT,1,males,15,24,2004.0,1.0,20.361498
3,1,Italy,IT,1,males,15,24,2004.0,4.0,22.135774
4,1,Italy,IT,1,males,15,24,2004.0,3.0,19.101695
...,...,...,...,...,...,...,...,...,...,...
95,1,Italy,IT,2,females,15,24,2004.0,Total,27.310948
96,1,Italy,IT,2,females,15,24,2004.0,2.0,28.927354
97,1,Italy,IT,2,females,15,24,2004.0,1.0,27.581100
98,1,Italy,IT,2,females,15,24,2004.0,4.0,28.376607


In [53]:
df.columns

Index(['Territory_ID', 'Territory', 'Territory_acr', 'Gender_ID', 'Gender',
       'ages from', 'ages to', 'Year', 'Quarter', 'Unemployment rate'],
      dtype='object')

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13680 entries, 0 to 13679
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Territory_ID       13680 non-null  int64  
 1   Territory          13680 non-null  object 
 2   Territory_acr      13680 non-null  object 
 3   Gender_ID          13680 non-null  int64  
 4   Gender             13680 non-null  object 
 5   ages from          13680 non-null  int64  
 6   ages to            13680 non-null  int64  
 7   Year               13680 non-null  float64
 8   Quarter            13680 non-null  object 
 9   Unemployment rate  13680 non-null  float64
dtypes: float64(2), int64(4), object(4)
memory usage: 1.0+ MB


In [55]:
# Drop columns not needed for EDA
df.drop(columns=['Territory_ID','Territory_acr','Gender_ID'], inplace=True)

In [56]:
# check for duplicates
df.duplicated().sum()

0

In [57]:
# Missing values
df.isnull().sum()

Territory            0
Gender               0
ages from            0
ages to              0
Year                 0
Quarter              0
Unemployment rate    0
dtype: int64

In [58]:
# Check for missing values
df['Year'] = df['Year'].astype('int')

In [59]:
df['Quarter'].value_counts()

Total    2736
2.0      2736
1.0      2736
4.0      2736
3.0      2736
Name: Quarter, dtype: int64

In [60]:
# Make a Pivot Table based on Quarters
df_pivot = pd.pivot_table(df, values='Unemployment rate', index=['Territory', 'ages from','Gender','ages to', 'Year'], columns=['Quarter'])

In [61]:
df_pivot = df_pivot.reset_index()

In [62]:
df_pivot.head(5)

Quarter,Territory,ages from,Gender,ages to,Year,1.0,2.0,3.0,4.0,Total
0,Centro (I),15,females,24,2004,28.270221,29.145744,20.403515,25.201805,25.647614
1,Centro (I),15,females,24,2005,28.015023,23.877914,26.554460,24.612646,25.755701
2,Centro (I),15,females,24,2006,28.145904,25.821481,22.532402,27.071081,25.927753
3,Centro (I),15,females,24,2007,23.704696,17.945794,19.927928,26.034254,21.957148
4,Centro (I),15,females,24,2008,21.346537,23.613878,25.277714,27.767465,24.463689


In [63]:
# melt the dataframe to combine 1.0, 2.0, 3.0 and 4.0 as one column while Total should be separate
df_melt = pd.melt(df_pivot, id_vars=['Territory', 'Gender', 'ages from', 'ages to', 'Year','Total'], 
                  value_vars=['1.0', '2.0', '3.0', '4.0'], var_name='Quarter', value_name='Unemployment rate')



In [64]:
df_melt.rename(columns={'Total':'Yearly_TotalUnemploymentRate'},inplace=True)

In [65]:
df_melt.head(5)

,Territory,Gender,ages from,ages to,Year,Yearly_TotalUnemploymentRate,Quarter,Unemployment rate
0,Centro (I),females,15,24,2004,25.647614,1.0,28.270221
1,Centro (I),females,15,24,2005,25.755701,1.0,28.015023
2,Centro (I),females,15,24,2006,25.927753,1.0,28.145904
3,Centro (I),females,15,24,2007,21.957148,1.0,23.704696
4,Centro (I),females,15,24,2008,24.463689,1.0,21.346537


In [66]:
# Seperate into regional and national
df_melt['Territory'].value_counts()

Centro (I)     1824
Italy          1824
Mezzogiorno    1824
Nord           1824
Nord-est       1824
Nord-ovest     1824
Name: Territory, dtype: int64

In [67]:
df_melt['Gender'].value_counts()

females    3648
males      3648
total      3648
Name: Gender, dtype: int64

In [68]:
# Seperate into National and regional
national_data = df_melt[df_melt['Territory'] == 'Italy']
regional_data = df_melt[df_melt['Territory'] != 'Italy']

In [69]:
national_data.head(5)

,Territory,Gender,ages from,ages to,Year,Yearly_TotalUnemploymentRate,Quarter,Unemployment rate
456,Italy,females,15,24,2004,27.310948,1.0,27.581100
457,Italy,females,15,24,2005,27.606126,1.0,28.600660
458,Italy,females,15,24,2006,25.461231,1.0,28.335586
459,Italy,females,15,24,2007,23.342217,1.0,23.514904
460,Italy,females,15,24,2008,24.682553,1.0,24.840614


In [70]:
regional_data.head(5)

,Territory,Gender,ages from,ages to,Year,Yearly_TotalUnemploymentRate,Quarter,Unemployment rate
0,Centro (I),females,15,24,2004,25.647614,1.0,28.270221
1,Centro (I),females,15,24,2005,25.755701,1.0,28.015023
2,Centro (I),females,15,24,2006,25.927753,1.0,28.145904
3,Centro (I),females,15,24,2007,21.957148,1.0,23.704696
4,Centro (I),females,15,24,2008,24.463689,1.0,21.346537


In [82]:
# Rename some of the column
national_data = national_data.rename(columns={'Territory':'Country','Unemployment rate':'National_UR'})

In [83]:
national_data.head(5)

,Country,Gender,ages from,ages to,Year,Yearly_TotalUnemploymentRate,Quarter,National_UR
456,Italy,females,15,24,2004,27.310948,1.0,27.581100
457,Italy,females,15,24,2005,27.606126,1.0,28.600660
458,Italy,females,15,24,2006,25.461231,1.0,28.335586
459,Italy,females,15,24,2007,23.342217,1.0,23.514904
460,Italy,females,15,24,2008,24.682553,1.0,24.840614


In [84]:
regional_data = regional_data.rename(columns={'Territory':'Region','Unemployment rate':'Regional_UR'})

In [85]:
merged_df = pd.merge(regional_data, national_data, on=['Gender', 'ages from', 'ages to', 'Year', 'Quarter'], how='outer', suffixes=('_R', '_N'))


In [86]:
merged_df = pd.merge(regional_data, national_data, on=['Gender', 'ages from', 'ages to', 'Year', 'Quarter'], how='outer',suffixes=('_R', '_N'))

In [87]:
merged_df.head(5)

,Region,Gender,ages from,ages to,Year,Yearly_TotalUnemploymentRate_R,Quarter,Regional_UR,Country,Yearly_TotalUnemploymentRate_N,National_UR
0,Centro (I),females,15,24,2004,25.647614,1.0,28.270221,Italy,27.310948,27.5811
1,Mezzogiorno,females,15,24,2004,44.857174,1.0,45.920425,Italy,27.310948,27.5811
2,Nord,females,15,24,2004,15.191595,1.0,13.594228,Italy,27.310948,27.5811
3,Nord-est,females,15,24,2004,14.552232,1.0,13.104420,Italy,27.310948,27.5811
4,Nord-ovest,females,15,24,2004,15.685978,1.0,13.968358,Italy,27.310948,27.5811


In [88]:
# Assuming merged_df is your DataFrame
merged_df = merged_df[['Country', 'Region', 'Gender', 'ages from', 'ages to', 'Year', 'Yearly_TotalUnemploymentRate_R', 'Yearly_TotalUnemploymentRate_N', 'Quarter', 'Regional_UR', 'National_UR']]

In [90]:
# Dropping the total rows from gender 
# We just want to the demographic information for analysis
# More helpful for analyzing crime rate

In [91]:
# Seperate it by Gender
gender_df = merged_df[merged_df['Gender'].isin(['males', 'females'])]

In [95]:
gender_df.head(5)

,Country,Region,Gender,ages from,ages to,Year,Yearly_TotalUnemploymentRate_R,Yearly_TotalUnemploymentRate_N,Quarter,Regional_UR,National_UR
0,Italy,Centro (I),females,15,24,2004,25.647614,27.310948,1.0,28.270221,27.5811
1,Italy,Mezzogiorno,females,15,24,2004,44.857174,27.310948,1.0,45.920425,27.5811
2,Italy,Nord,females,15,24,2004,15.191595,27.310948,1.0,13.594228,27.5811
3,Italy,Nord-est,females,15,24,2004,14.552232,27.310948,1.0,13.104420,27.5811
4,Italy,Nord-ovest,females,15,24,2004,15.685978,27.310948,1.0,13.968358,27.5811


In [97]:
gender_df.to_csv('quarterlyunemploymentbygender.csv', index=False)